In [15]:
# for database connections
from sqlalchemy import create_engine, inspect, text

# data
import pandas as pd

# Postgres credentials
import config as cfg

In [2]:
# Setup the SQL connection variables
SQL_USERNAME = cfg.SQL_USERNAME
SQL_PASSWORD = cfg.SQL_PASSWORD
SQL_IP = cfg.SQL_IP
SQL_PORT = cfg.SQL_PORT
DATABASE = cfg.DATABASE

In [3]:
# Connect to PostgreSQL server
connection_string = f'postgresql+psycopg2://{SQL_USERNAME}:{SQL_PASSWORD}@{SQL_IP}:{SQL_PORT}/{DATABASE}'
engine = create_engine(connection_string)

## upload campaign, category, subcategory and contact tables

In [4]:
# Check to see if the category and subcategory tables are in the database
inspector = inspect(engine)
print(inspector.get_table_names())

['contact', 'campaign', 'category', 'subcategory']


In [5]:
# Display all the columns of the tables in PostgreSQL to make sure they have the correct columns prior to loading the data

# Collect the names of tables within the database
tables = inspector.get_table_names()

# Using the inspector to print the column names within each table and its types
for table in tables:
    print(f"Table name: {table}")
    columns = inspector.get_columns(table)
    for column in columns:
        print(column["name"], column["type"])
        
    print()

Table name: contact
contact_id INTEGER
first_name VARCHAR(30)
last_name VARCHAR(30)
email VARCHAR(50)
last_update TIMESTAMP

Table name: campaign
funding_id INTEGER
contact_id INTEGER
company_name VARCHAR(50)
description VARCHAR(75)
goal NUMERIC
pledged NUMERIC
outcome VARCHAR(10)
backers_count INTEGER
country VARCHAR(2)
currency VARCHAR(3)
launched_date TIMESTAMP
end_date TIMESTAMP
staff_pick BOOLEAN
spotlight BOOLEAN
category_id VARCHAR(10)
subcategory_id VARCHAR(12)
last_update TIMESTAMP

Table name: category
category_id VARCHAR(10)
category VARCHAR(50)
last_update TIMESTAMP

Table name: subcategory
subcategory_id VARCHAR(12)
subcategory VARCHAR(50)
last_update TIMESTAMP



In [6]:
# Open category csv file and read it into a pandas dataframe
category_df = pd.read_csv('DB_Input/category.csv')
category_df.head()

,category_id,category
0,cat1,food
1,cat2,music
2,cat3,technology
3,cat4,theater
4,cat5,film & video


In [7]:
# Open subcategory csv file and read it into a pandas dataframe
subcategory_df = pd.read_csv('DB_Input/subcategory.csv')
subcategory_df.head()

,subcategory_id,subcategory
0,subcat1,food trucks
1,subcat2,rock
2,subcat3,web
3,subcat4,plays
4,subcat5,documentary


In [8]:
# Open contact csv file and read it into a pandas dataframe
contact_df = pd.read_csv('DB_Input/contact.csv')
contact_df.head()

,contact_id,first_name,last_name,email
0,4661,Cecilia,Velasco,cecilia.velasco@rodrigues.fr
1,3765,Mariana,Ellis,mariana.ellis@rossi.org
2,4187,Sofie,Woods,sofie.woods@riviere.com
3,4941,Jeanette,Iannotti,jeanette.iannotti@yahoo.com
4,2199,Samuel,Sorgatz,samuel.sorgatz@gmail.com


In [9]:
# Open campaign csv file and read it into a pandas dataframe
campaign_df = pd.read_csv('DB_Input/campaign.csv')
campaign_df.head()

,funding_id,contact_id,company_name,description,goal,pledged,outcome,backers_count,country,currency,launched_date,end_date,staff_pick,spotlight,category_id,subcategory_id
0,147,4661,"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100.0,0.0,failed,0,CA,CAD,2020-02-13 06:00:00,2021-03-01 06:00:00,False,False,cat1,subcat1
1,1175,2288,Werner-Bryant,Virtual uniform frame,1800.0,7991.0,successful,222,US,USD,2020-06-20 05:00:00,2021-01-30 06:00:00,False,False,cat1,subcat1
2,873,2067,Stewart LLC,Cloned bi-directional architecture,1300.0,12047.0,successful,113,US,USD,2020-11-29 06:00:00,2021-06-11 05:00:00,False,False,cat1,subcat1
3,2568,5989,Castillo-Carey,Cross-platform solution-oriented process impro...,142400.0,21307.0,failed,296,US,USD,2020-05-05 05:00:00,2021-05-30 05:00:00,False,False,cat1,subcat1
4,1211,3307,"Wright, Hartman and Yu",User-friendly tertiary array,3300.0,12437.0,successful,131,US,USD,2021-01-29 06:00:00,2021-02-13 06:00:00,False,False,cat1,subcat1


In [10]:
# If category table exists in the database, load the category data into the table
if 'category' in inspector.get_table_names():
    category_df.to_sql('category', schema='public', con=engine, index=False, if_exists='append', method='multi')

In [11]:
# If subcategory table exists in the database, load the subcategory data into the table
if 'subcategory' in inspector.get_table_names():
    subcategory_df.to_sql('subcategory', schema='public', con=engine, index=False, if_exists='append', method='multi')

In [12]:
# If contact table exists in the database, load the contact data into the table
if 'contact' in inspector.get_table_names():
    contact_df.to_sql('contact', schema='public', con=engine, index=False, if_exists='append', method='multi')

In [13]:
# If campaign table exists in the database, load the campaign data into the table
if 'campaign' in inspector.get_table_names():
    campaign_df.to_sql('campaign', schema='public', con=engine, index=False, if_exists='append', method='multi')

In [16]:
# Verify count from campaign table after load
query = """
		SELECT
			count(*)
		FROM
			campaign
		;
		"""

campaign_count_after_import_df = pd.read_sql(text(query), con=engine)
campaign_count_after_import_df.head()

,count
0,1000


In [17]:
# Verify count from category table after load
query = """
		SELECT
			count(*)
		FROM
			category
		;
		"""

category_count_after_import_df = pd.read_sql(text(query), con=engine)
category_count_after_import_df.head()

,count
0,9


In [18]:
# Verify count from subcategory table after load
query = """
		SELECT
			count(*)
		FROM
			subcategory
		;
		"""

subcategory_count_after_import_df = pd.read_sql(text(query), con=engine)
subcategory_count_after_import_df.head()

,count
0,24


In [19]:
# Verify count from contact table after load
query = """
		SELECT
			count(*)
		FROM
			contact
		;
		"""

contact_count_after_import_df = pd.read_sql(text(query), con=engine)
contact_count_after_import_df.head()

,count
0,1000
